In [ ]:
import json
import pandas as pd
import geopandas as gpd

import plotly.express as px

from covis.utils import get_project_root

# regional choropleth with date slider 

In [ ]:
weekly_reg = pd.read_csv(
    get_project_root() / "output/covid19_weekly_regional_deaths_2020-2022.csv",
    index_col=0
)

In [ ]:
print(weekly_reg.shape)
weekly_reg.head()

In [ ]:
# map regions:
jpath = get_project_root() / "output/eer_dict.json"
with open(jpath) as f:
    eer = json.load(f)

eer

In [ ]:
geo = gpd.read_parquet(
    get_project_root() / "output/EWgeo.pkl",
)

In [ ]:
geo

In [ ]:
geo_json = geo[["EER13NM", "geometry"]]
geo_json.set_index("EER13NM", inplace=True)
geoj = geo_json.to_dict()

In [ ]:
geo_json

In [ ]:
geoj

In [ ]:
geo = geo.set_crs("EPSG:27700")

In [ ]:
weekly_reg

In [ ]:
weekly_reg_long = weekly_reg.melt(
    id_vars="Week ended",
    value_vars=[col for col in weekly_reg.columns if col not in ["Week ended"]]
)

In [ ]:
weekly_reg_long.head()

In [ ]:
geoj

In [ ]:
test = weekly_reg_long[weekly_reg_long["variable"] == "West Midlands"]

In [ ]:
geo_json.crs

In [ ]:
geo_json["geometry"]["West Midlands"]

In [ ]:
# weekly_reg_long.sort_values("Week ended").reset_index().loc[0:50]

In [ ]:
weekly_reg_long_test = weekly_reg_long.sort_values("Week ended").reset_index().loc[100:300]

In [ ]:
fig = px.choropleth(
    weekly_reg_long_test,
    geojson=geo_json["geometry"],
    locations=weekly_reg_long_test.variable,
    # center={"lat": 55.09621, "lon": -4.0286298},
    # scope="europe",
    color=weekly_reg_long_test.value,
    animation_frame="Week ended",
    color_continuous_scale="reds",
    range_color=(weekly_reg_long_test.value.min(), weekly_reg_long_test.value.max())
    # zmin=weekly_reg_long_test.value.min(),
    # zmax=weekly_reg_long_test.value.max()
)

fig.update_geos(
    fitbounds="geojson",
    visible=False
)

# fig.show()

In [ ]:
fig.write_html(
    get_project_root() / "output/chroro_slider.html"
)